## Extra convenience methods for common responses

### OPeNDAP

In [1]:
from erddapy import ERDDAP


e = ERDDAP(server="https://gliders.ioos.us/erddap")

In [2]:
from netCDF4 import Dataset


e.constraints = None
e.protocol = "tabledap"
e.dataset_id = "whoi_406-20160902T1700"

opendap_url = e.get_download_url(
    response="opendap",
)

print(opendap_url)
with Dataset(opendap_url) as nc:
    print(nc.summary)

https://gliders.ioos.us/erddap/tabledap/whoi_406-20160902T1700
Slocum glider dataset gathered as part of the TEMPESTS (The Experiment to Measure and Predict East coast STorm Strength), funded by NOAA through CINAR (Cooperative Institute for the North Atlantic Region).


### netCDF Climate and Forecast

In [3]:
e.response = "nc"
e.variables = ["longitude", "latitude", "temperature", "salinity"]

nc = e.to_ncCF()

print(nc.Conventions)
print(nc["temperature"])

Unidata Dataset Discovery v1.0, COARDS, CF-1.6
<class 'netCDF4._netCDF4.Variable'>
float32 temperature(obs)
    _ChunkSizes: 73
    _FillValue: -999.0
    actual_range: [ 9.9473 22.6177]
    ancillary_variables: qartod_temperature_climatological_flag qartod_temperature_flat_line_flag qartod_temperature_gross_range_flag qartod_temperature_rate_of_change_flag qartod_temperature_spike_flag temperature_qc
    colorBarMaximum: 32.0
    colorBarMinimum: 0.0
    coordinates: time latitude longitude depth
    instrument: instrument_ctd
    ioos_category: Temperature
    long_name: Sea Water Temperature
    observation_type: measured
    platform: platform
    standard_name: sea_water_temperature
    units: Celsius
    valid_max: 40.0
    valid_min: -5.0
unlimited dimensions: 
current shape = (54245,)
filling on


### xarray

In [4]:
ds = e.to_xarray(decode_times=False)

ds

<xarray.Dataset>
Dimensions:          (obs: 54245, profile: 525, trajectory: 1)
Coordinates:
  * trajectory       (trajectory) object 'whoi_406-20160902T1700'
    time             (profile) float64 ...
    longitude        (profile) float64 ...
    latitude         (profile) float64 ...
    depth            (obs) float32 ...
Dimensions without coordinates: obs, profile
Data variables:
    profile_id       (profile) float64 ...
    trajectoryIndex  (profile) int32 ...
    rowSize          (profile) int32 ...
    temperature      (obs) float32 ...
    salinity         (obs) float32 ...
Attributes:
    acknowledgment:                This deployment supported by NOAA through ...
    cdm_data_type:                 TrajectoryProfile
    cdm_profile_variables:         time_uv,lat_uv,lon_uv,u,v,profile_id,time,...
    cdm_trajectory_variables:      trajectory,wmo_id
    comment:                       Glider operatored by Woods Hole Oceanograp...
    contributor_name:              Robert Todd, Larry George, Patrick Deane, ...
    contributor_role:              Principal Investigator, pilot and technici...
    Conventions:                   Unidata Dataset Discovery v1.0, COARDS, CF...
    creator_email:                 rtodd@whoi.edu
    creator_name:                  Robert Todd
    creator_url:                   gliders.whoi.edu
    date_created:                  2016-09-20T10:06:04Z
    date_issued:                   2016-09-20T10:06:04Z
    Easternmost_Easting:           -70.67537291773046
    featureType:                   TrajectoryProfile
    format_version:                /tmp/IOOS_Glider_NetCDF_v2.0-qartod.nc
    geospatial_lat_max:            41.120018298037856
    geospatial_lat_min:            40.257794149155615
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -70.67537291773046
    geospatial_lon_min:            -71.54982816987487
    geospatial_lon_units:          degrees_east
    geospatial_vertical_max:       98.05
    geospatial_vertical_min:       2.49
    geospatial_vertical_positive:  down
    geospatial_vertical_units:     m
    gts_ingest:                    true
    history:                       2016-09-20T10:06:04Z /home/kerfoot/slocum/...
    id:                            whoi_406-20160920T1329
    infoUrl:                       https://gliders.ioos.us/erddap/
    institution:                   Woods Hole Oceanographic Institution
    ioos_dac_checksum:             6e67c5e2c364def562bb32242dec8fd5
    ioos_dac_completed:            True
    keywords:                      AUVS > Autonomous Underwater Vehicles, Ear...
    keywords_vocabulary:           GCMD Science Keywords
    license:                       This data may be redistributed and used wi...
    Metadata_Conventions:          Unidata Dataset Discovery v1.0, COARDS, CF...
    naming_authority:              edu.rutgers.marine
    Northernmost_Northing:         41.120018298037856
    platform_type:                 Slocum Glider
    project:                       TEMPESTS
    publisher_email:               kerfoot@marine.rutgers.edu
    publisher_name:                John Kerfoot
    publisher_url:                 https://rucool/marine.rutgers.edu
    references:                    https://ioos.noaa.gov/wp-content/uploads/2...
    sea_name:                      Mid-Atlantic Bight
    source:                        Observational data from a profiling glider
    sourceUrl:                     (local files)
    Southernmost_Northing:         40.257794149155615
    standard_name_vocabulary:      CF-v25
    subsetVariables:               trajectory,wmo_id,time_uv,lat_uv,lon_uv,u,...
    summary:                       Slocum glider dataset gathered as part of ...
    time_coverage_end:             2016-09-20T13:32:33Z
    time_coverage_start:           2016-09-02T17:05:38Z
    title:                         whoi_406-20160902T1700
    Westernmost_Easting:           -71.54982816987487

### iris

In [5]:
import warnings

# Iris warnings are quire verbose!
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    cubes = e.to_iris()

print(cubes)

0: Trajectory Name / (1)               (-- : 1)
1: Number of Observations for this Profile / (1) (-- : 525)
2: sea_water_practical_salinity / (1)  (-- : 54245)
3: Profile ID / (1)                    (-- : 525)
4: The trajectory to which this profile is associated. / (1) (-- : 525)
5: sea_water_temperature / (Celsius)   (-- : 54245)
6: latitude / (degrees)                (-- : 525)
7: longitude / (degrees)               (-- : 525)
8: time / (seconds since 1970-01-01T00:00:00Z) (-- : 525)


In [6]:
cubes.extract_strict("sea_water_temperature")

Sea Water Temperature (Celsius),--
Shape,54245
Auxiliary coordinates,
depth,x
Attributes,
Conventions,"Unidata Dataset Discovery v1.0, COARDS, CF-1.6"
Easternmost_Easting,-70.67537291773046
Metadata_Conventions,"Unidata Dataset Discovery v1.0, COARDS, CF-1.6"
Northernmost_Northing,41.120018298037856
Southernmost_Northing,40.257794149155615
Westernmost_Easting,-71.54982816987487
